In [ ]:
%%capture
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score
!pip install evaluate

In [ ]:
import numpy as np
import pandas as pd
import evaluate
from datasets import load_dataset
from transformers import pipeline
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
translator = pipeline("translation", model='/working/my/')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
file_path = 'input/test-news/translated_news_data.xlsx'
df = pd.read_excel(file_path)

df

,text,language,translation
0,في اليوم الـ56 من الحرب على غزة، استأنفت إسرائ...,ar,На 56 -й день войны с Газой Израиль возобновил...
1,سقط عشرات الفلسطينيين بين شهيد ومصاب في غارات ...,ar,Десятки палестинцев упали между мучеником и ра...
2,قال رئيس الوزراء الإسرائيلي بنيامين نتنياهو إن...,ar,Премьер -министр Израиля Бенджамин Нетаньяху с...
3,طرح رئيس الوزراء الإسرائيلي السابق إيهود أولمر...,ar,Бывший премьер -министр Израиля Эхуд Олмерт пр...
4,قالت مجلة إيكونوميست البريطانية إن من الواضح أ...,ar,"Журнал британского экономиста сказал, что ясно..."
...,...,...,...
145,Les inondations après les fortes pluies sont t...,fr,Наводнения после проливных дождей очень важны ...
146,"Etincelles, feux d’artifice, spectacles de feu...",fr,"Sparks, Fireworks, Fire Shows и Lights, Sainte..."
147,"Le jeune supporter, tué par un tir en marge d'...",fr,"Молодой сторонник, убитый выстрелом в кулуарах..."
148,"Une voiture a fait une sortie de route, sur la...",fr,"Автомобиль совершил поездку в город Порта, чет..."


In [ ]:
text_en = df[df['language'] == 'en']['text'].values.tolist()
text_translate_en = df[df['language'] == 'en']['translation'].values.tolist()

In [ ]:
text_fr = df[df['language'] == 'fr']['text'].values.tolist()
text_translate_fr = df[df['language'] == 'fr']['translation'].values.tolist()

In [ ]:
text_ar = df[df['language'] == 'ar']['text'].values.tolist()
text_translate_ar = df[df['language'] == 'ar']['translation'].values.tolist()

In [ ]:
results_en = translator(text_en, src_lang='en', tgt_lang='ru')

In [ ]:
results_fr = translator(text_fr, src_lang='fr', tgt_lang='ru')

In [ ]:
results_ar = translator(text_ar, src_lang='ar', tgt_lang='ru')

In [ ]:
res_en = []
for seq in results_en:
    res_en.append(seq['translation_text'])

res_fr = []
for seq in results_fr:
    res_fr.append(seq['translation_text'])

res_ar = []
for seq in results_ar:
    res_ar.append(seq['translation_text'])

In [ ]:
res_en[0]

'Смертельная борьба разрвалась по сектору Газа в пятницу, когда израильские военные возобновили свои боевые операции после временного вмешательства с Хамасом, которое проложило путь к неделе освобождения заложников.'

In [ ]:
text_en[0]

'CNN — Deadly fighting broke out across Gaza on Friday as Israel’s military resumed combat operations after the temporary truce with Hamas that had paved the way for a week of hostage releases broke down. Smoke could be seen billowing across parts of the densely populated enclave as Israel’s military'

In [ ]:
bleu = metric_bleu.compute(predictions=res_en, references=text_translate_en)
print("BLEU: ", bleu["score"])

rouge = metric_rouge.compute(predictions=res_en, references=text_translate_en)
print("ROUGE1", rouge['rouge1'])
print("ROUGE2", rouge['rouge2'])

BLEU:  27.684694952040545
ROUGE1 0.5067460317460319
ROUGE2 0.16558008658008658


In [ ]:
bleu = metric_bleu.compute(predictions=res_fr, references=text_translate_fr)
print("BLEU: ", bleu["score"])

rouge = metric_rouge.compute(predictions=res_fr, references=text_translate_fr)
print("ROUGE1", rouge['rouge1'])
print("ROUGE2", rouge['rouge2'])

BLEU:  16.995431856473008
ROUGE1 0.4727507106714424
ROUGE2 0.2016883116883117


In [ ]:
bleu = metric_bleu.compute(predictions=res_ar, references=text_translate_ar)
print("BLEU: ", bleu["score"])

rouge = metric_rouge.compute(predictions=res_ar, references=text_translate_ar)
print("ROUGE1", rouge['rouge1'])
print("ROUGE2", rouge['rouge2'])

BLEU:  20.413413614608253
ROUGE1 0.3516176046176046
ROUGE2 0.1341904761904762


In [ ]:
en_df = pd.DataFrame(list(zip(res_en, text_translate_en, text_en)), columns =['candidate', 'reference', 'original'])

In [ ]:
fr_df = pd.DataFrame(list(zip(res_fr, text_translate_fr, text_fr)), columns =['candidate', 'reference', 'original'])

In [ ]:
ar_df = pd.DataFrame(list(zip(res_ar, text_translate_ar, text_ar)), columns =['candidate', 'reference', 'original'])

In [ ]:
en_df.to_excel('eng_news.xlsx', index=False)
fr_df.to_excel('fr_news.xlsx', index=False)
ar_df.to_excel('ar_news.xlsx', index=False)